# Adversarial Search: Playing Connect 4


## Instructions

Total Points: Undegraduates 10, graduate students 11

Complete this notebook and submit it. The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

You will implement different versions of agents that play Connect 4:

> "Connect 4 is a two-player connection board game, in which the players choose a color and then take turns dropping colored discs into a seven-column, six-row vertically suspended grid. The pieces fall straight down, occupying the lowest available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own discs." (see [Connect Four on Wikipedia](https://en.wikipedia.org/wiki/Connect_Four))

## Task 1: Defining the Search Problem [1 point]

Define the components of the search problem:

* Initial state - The initial state is 
* Actions - A column number for where the player drops their playing piece. It will fall to the lowest open place for that column.
* Transition model - Place a piece in the column that was selected by the player
* Goal state - Create 4 pieces in a row



* **Initial state** - The initial state is 
* **Actions** - A column number for where the player drops their playing piece. It will fall to the lowest open place for that column.
* **Transition model** - Place a piece in the column that was selected by the player
* **Goal state** - Create 4 pieces in a row

How big is the search space?

TODO: calculate this state space

__Note:__ The search space for a $6 \times 7$ board is large. You can experiment with smaller boards (the smallest is $4 times \4$) and/or changing the winning rule to connect 3 instead of 4.

## Task 2: Game Environment and Random Agent [2 point]

Use a numpy character array as the board.

In [2]:
import numpy as np

def empty_board(shape=(6, 7)):
    return np.full(shape=shape, fill_value=' ')

def fill_board(board):
    s = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    i = 0
    for row in range(len(board)):
        for col in range(len(board[0])):
            board[row][col] = s[i%len(s)]
            i+=1

def pretty_board(board):
    for i in range(0,board.shape[1]+board.shape[1]+1):
        print('-',end='')
    print()
    for row in board:
        print('|', end='')
        for piece in row:
            print(piece, end='')
            print('|',end='')
        print() 
    for i in range(0,board.shape[1]+board.shape[1]+1):
        print('-',end='')
    print()            
            
print(empty_board())

[[' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ']]


Instead of colors for the players use 'x' and 'o' to represent the players. Make sure that your agent functions all have the from: `agent_type(board, player = 'x')`, where board is the current board position and player is the player whose next move it is and who the agent should play.

Implement the board and helper functions for:

* The transition model (result).
* The utility function.
* Check for terminal states.
* A check for available actions.
* A function to visualize the board.

Make sure that all these functions work with boards of different sizes.

Implement an agent that plays randomly and let two random agents play against each other 1000 times. How often does each player win? Is the result expected? 

In [19]:
import random

def flip_player(player):
    if player == 'x':
        return 'o'
    else:
        return 'x'
    
def actions(board):
    options = []
    for col in range(0, len(board[0])):
        if board[0][col] == ' ':
            options.append(col)
    return options

def agent_random(board, player = 'x'):
    options = actions(board)
    if len(options) == 0:
        return None
    return random.choice(options)
            
# Environment methods
def place_piece(board, col, player='x'):
    """Place the player piece in the col on the board"""
    if board[0][col] != ' ':
        return False
    
    next_open_row = len(board) - 1 # if the col is empty use the bottom row
    for row in range(0, len(board)):
        if board[row][col] != ' ':
            next_open_row = row - 1
            break
            
    board[next_open_row][col] = player


def result(board, player, action):
    board_copy = board.copy()
    place_piece(board_copy, action, player)
    return board_copy


# TODO: Optimize this
def check_series_for_winner(series):
    """Check if this line (could be anywhere)
    has 4 in a row"""
    count = 0
    prev = ' '
    for current_space in series:
        if current_space != ' ' and current_space == prev:
            count += 1
        elif current_space != ' ' and current_space != prev:
            count = 1
            prev = current_space
        elif current_space == ' ':
            prev = ' '
            count = 0
        
        if count == 4:
            return prev

def get_all_series(board):
    """Get all of the rows/cols/diags that could be winning streaks"""
    num_cols = len(board[0])
    num_rows = len(board)
    
    runs = []
    for row in range(0, num_rows):
        runs.append(board[row])
    for col in range(0, num_cols):
        runs.append(board[:,col])
        
    for offset in range(1-num_rows, num_cols):
        runs.append(np.diagonal(board, offset=offset))
        runs.append(np.diagonal(np.fliplr(board), offset=offset))
    # get all diags
    max_row_cols = max(num_rows, num_cols)

    return runs

def terminal(board):
    """Determine if a player has won"""
    num_cols = len(board[0])
    num_rows = len(board)
    if len(actions(board)) == 0:
        return 'c'
    
    series = get_all_series(board)
    for s in series:
        winner = check_series_for_winner(s)
        if winner is not None:
            return winner
    return ' '

def utility(board, player = 'x'):
    """check is a state a terminal state, return the utility if it is.
    None means not terminal"""
    
    winner = terminal(board)
    if winner == player:
        return +1
    if winner == flip_player(player):
        return -1
    if winner == 'c':
        return 0
    return None

def play_list_plays(moves, board, starting_player='x'):
    for move in moves:
        place_piece(board=board, player=starting_player, col=move)
        starting_player = flip_player(starting_player)

In [20]:

def play_game(player_x, player_o, board_shape = (6, 7), verbose=False):
    players_turn = 'x'
    
    board = empty_board(board_shape)
    while True:
        col = -1
        if players_turn == 'x':
            col = player_x(board, players_turn)
        else:
            col = player_o(board, players_turn)
        #place_piece(board, col, players_turn)
        board = result(board=board, player=players_turn, action=col)
        if verbose:
            print('Player: ', players_turn, 'placing in col', col)
        winner = terminal(board)
        if winner == 'c':
            return ('c', board)
        elif winner != ' ':
            return (players_turn, board)
        
        players_turn = flip_player(players_turn)


### Test Utility

In [21]:
board = empty_board()
pretty_board(board)
print('x:', utility(board, player='x'), 'o:', utility(board, player='o'))
play_list_plays([3,2,3,1,3,0,3], board)
pretty_board(board)
print('x:', utility(board, player='x'), 'o:', utility(board, player='o'))
board = empty_board()
play_list_plays([0,1,1,2,3,2,2,5,3,3,3], board)
pretty_board(board)
print('x:', utility(board, player='x'), 'o:', utility(board, player='o'))
board = empty_board()
play_list_plays([0,1,1,2,3,2,2,5,3,3,3], board, starting_player='o')
pretty_board(board)
print('x:', utility(board, player='x'), 'o:', utility(board, player='o'))
board = empty_board(shape = (5,6))
play_list_plays([2,2,1,1,0,1,1,2,2,0,0], board)
print('x:', utility(board, player='x'), 'o:', utility(board, player='x'))

---------------
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
---------------
x: None o: None
---------------
| | | | | | | |
| | | | | | | |
| | | |x| | | |
| | | |x| | | |
| | | |x| | | |
|o|o|o|x| | | |
---------------
x: 1 o: -1
---------------
| | | | | | | |
| | | | | | | |
| | | |x| | | |
| | |x|o| | | |
| |x|o|x| | | |
|x|o|o|x| |o| |
---------------
x: 1 o: -1
---------------
| | | | | | | |
| | | | | | | |
| | | |o| | | |
| | |o|x| | | |
| |o|x|o| | | |
|o|x|x|o| |x| |
---------------
x: -1 o: 1
x: None o: None


### Random V Random

In [22]:
x_ct = 0
o_ct = 0
c_ct = 0
games = []
for i in range(1000):
    game = play_game(agent_random, agent_random, verbose=False)
    winner = game[0]
    if winner == 'x':
        x_ct += 1
    elif winner == 'o':
        o_ct += 1
    elif winner == 'c':
        c_ct += 1
    games.append(game)
print('X won:', x_ct, '|O won', o_ct, '|Cat games', c_ct)

X won: 546 |O won 454 |Cat games 0


## Task 3: Minimax Search with Alpha-Beta Pruning [4 points]

### Implement the search starting from a given board and specifying the player.

In [31]:
import math
import random
DEBUG = 1 # 1 ... count nodes, 2 ... debug each node
COUNT = 0
def alpha_beta_search(board, player='x'):
    global DEBUG, COUNT
    COUNT = 0
    
    value, move = max_value_ab(board, player, -math.inf, +math.inf)
    
    if DEBUG >= 1: print(f"Number of nodes searched: {COUNT}") 
        
    return value, move

def max_value_ab(board, player, alpha, beta):
    """Player's best move"""
    global DEBUG, COUNT
    COUNT += 1
    
    v = utility(board, player)
    if DEBUG >= 2: print("max: \n" + str(board) + str([alpha, beta, v])) 
    if v is not None: return v, None
    
    v, move = -math.inf, None
    
    moves = actions(board)
    random.shuffle(moves)
    for a in moves:
        v2, a2 = min_value_ab(result(board, player, a), player, alpha, beta)
        if v2 > v:
            v, move = v2, a
            alpha = max(alpha, v)
        if v >= beta:
            return v, move
    return v, move

def min_value_ab(board, player, alpha, beta):
    """opponent's best response"""
    global DEBUG, COUNT
    COUNT += 1
    
    #return utility if state is terminal state
    v = utility(board, player)
    if DEBUG >= 2: print("min: \n" + str(board) + str([alpha, beta, v]))
    if v is not None: return v, None
    
    v, move = +math.inf, None
    
    moves = actions(board)
    random.shuffle(moves)
    for a in moves:
        v2, a2 = max_value_ab(result(board, flip_player(player), a),player,alpha,beta)
        if v2 < v:
            v, move = v2, a
            beta = min(beta, v)
        if v <= alpha:
            return v, move
        
    return v, move

### Test against Random

In [32]:
def mini_max_ab_agent(board, player='x'):
    v, col = alpha_beta_search(board=board)
    return col
#
#def play_agents(agent_x, agent_o, shape=(6,7)):
#    global DEBUG
#    DEBUG = 2
#    board = empty_board(shape=shape)
#    player = 'x'
#    finised = False
#    while not finished:
#        if player == 'x':
#            col = agent_x(board, player=player)
#            if DEBUG == 2:
#                print(player, 'placing in col', col)
#            pretty_board(board)
#            place_piece(board, col, player=player)
#        else:
#            col = agent_o(board, player=player)
#            if DEBUG == 2:
#                print(player, 'placing in col', col)
#            pretty_board(board)
#            place_piece(board, col, player=player)
#        player = flip_player(player)
#        finished = utility(board, player) is None
#    pretty_board(board)
#play_agents(agent_x=mini_max_ab_agent, agent_o= shape=(4,4))

play_game(mini_max_ab_agent, agent_random, board_shape=(4,4), verbose=True)

Number of nodes searched: 87099
Player:  x placing in col 2
Player:  o placing in col 2
Number of nodes searched: 20985
Player:  x placing in col 2
Player:  o placing in col 2
Number of nodes searched: 2119
Player:  x placing in col 1
Player:  o placing in col 1
Number of nodes searched: 501
Player:  x placing in col 0
Player:  o placing in col 1
Number of nodes searched: 112
Player:  x placing in col 3


('x',
 array([[' ', ' ', 'o', ' '],
        [' ', 'o', 'x', ' '],
        [' ', 'o', 'o', ' '],
        ['x', 'x', 'x', 'x']], dtype='<U1'))

In [35]:
board = empty_board(shape = (4,4))
display(alpha_beta_search(board, player='x'))

Number of nodes searched: 100018


(0, 2)

Experiment with some manually created boards (at least 5) to check if the agent spots wining opportunities.

In [36]:
%%time
board = empty_board(shape = (4,5))
place_piece(board, col=2, player= 'x')
place_piece(board, col=2, player= 'o')
place_piece(board, col=1, player= 'x')
place_piece(board, col=1, player= 'o')
place_piece(board, col=0, player= 'x')
place_piece(board, col=1, player= 'o')
place_piece(board, col=1, player= 'x')
place_piece(board, col=2, player= 'o')
pretty_board(board)
DEBUG = 0
display(alpha_beta_search(board, player='x'))

-----------
| |x| | | |
| |o|o| | |
| |o|o| | |
|x|x|x| | |
-----------


(1, 3)

Wall time: 210 ms


In [37]:
%%time
board = empty_board(shape = (5,5))
place_piece(board, col=0, player= 'x')
place_piece(board, col=1, player= 'o')
place_piece(board, col=1, player= 'x')
place_piece(board, col=2, player= 'o')
place_piece(board, col=3, player= 'x')
place_piece(board, col=3, player= 'o')
place_piece(board, col=2, player= 'x')
place_piece(board, col=3, player= 'o')
pretty_board(board)
display(alpha_beta_search(board, player='x'))

-----------
| | | | | |
| | | | | |
| | | |o| |
| |x|x|o| |
|x|o|o|x| |
-----------


(0, 1)

Wall time: 2min 37s


In [ ]:
%%time
board = empty_board(shape = (5,6))
place_piece(board, col=2, player= 'x')
place_piece(board, col=2, player= 'o')
place_piece(board, col=1, player= 'x')
place_piece(board, col=1, player= 'o')
place_piece(board, col=0, player= 'x')
place_piece(board, col=1, player= 'o')
place_piece(board, col=1, player= 'x')
place_piece(board, col=2, player= 'o')
pretty_board(board)
DEBUG = 0
display(alpha_beta_search(board, player='x'))

-------------
| | | | | | |
| |x| | | | |
| |o|o| | | |
| |o|o| | | |
|x|x|x| | | |
-------------


In [23]:
%%time
board = empty_board(shape = (5,6))
place_piece(board, col=2, player= 'o')
place_piece(board, col=2, player= 'x')
place_piece(board, col=1, player= 'o')
place_piece(board, col=1, player= 'x')
place_piece(board, col=0, player= 'o')
place_piece(board, col=1, player= 'x')
place_piece(board, col=1, player= 'o')
place_piece(board, col=2, player= 'x')
place_piece(board, col=2, player= 'o')
pretty_board(board)
DEBUG = 0
display(alpha_be
        ta_search(board, player='x'))

-------------
| | | | | | |
| |o|o| | | |
| |x|x| | | |
| |x|x| | | |
|o|o|o| | | |
-------------


(1, 3)

Wall time: 5min 43s


In [25]:
%%time
board = empty_board(shape = (5,6))
place_piece(board, col=2, player= 'x')
place_piece(board, col=2, player= 'o')
place_piece(board, col=1, player= 'x')
place_piece(board, col=1, player= 'o')
place_piece(board, col=0, player= 'x')
place_piece(board, col=1, player= 'o')
place_piece(board, col=1, player= 'x')
place_piece(board, col=2, player= 'o')
place_piece(board, col=2, player= 'x')
place_piece(board, col=0, player= 'o')
place_piece(board, col=0, player= 'x')
pretty_board(board)
DEBUG = 0
display(alpha_beta_search(board, player='x'))

-------------
| | | | | | |
| |x|x| | | |
|x|o|o| | | |
|o|o|o| | | |
|x|x|x| | | |
-------------


(1, 0)

Wall time: 1min 18s


How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [ ]:
# Your code/ answer goes here.
board = empty_board(shape=(5,4))
display(alpha_beta_search(board, player='x'))

### Move ordering

Describe and implement a simple move ordering strategy. How does this strategy influence the time it takes to 
make a move?

In [8]:
# Your code/ answer goes here.

### Playtime

Let the Minimax Search agent play a random agent on a small board. Analyze wins, losses and draws.

In [9]:
# Your code/ answer goes here.

## Task 4: Heuristic Alpha-Beta Tree Search [3 points] 

### Heuristic evaluation function

Define and implement a heuristic evaluation function.

In [10]:
# Your code/ answer goes here.

### Cutting off search 

Modify your Minimax Search with Alpha-Beta Pruning to cut off search at a specified depth and use the heuristic evaluation function. Experiment with different cutoff values.

In [11]:
# Your code/ answer goes here.

Experiment with the same manually created boards as above to check if the agent spots wining opportunities.

In [12]:
# Your code/ answer goes here.

How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [13]:
# Your code/ answer goes here.

### Forward Pruning

Add forward pruning to the cutoff search where you do not consider moves that have a low evaluation value after a shallow search 
(way smaller than the cuttoff value).

In [14]:
# Your code/ answer goes here.

How long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [15]:
# Your code/ answer goes here.

### Playtime

Let two heuristic search agents (different cutoff depth, different heuristic evaluation function or different forward pruning) compete against each other on a reasonably sized board. Since there is no randomness, you only need to let them play once.

In [16]:
# Your code/ answer goes here.

## Challenge task [+ 1 bonus point]

Find another student and let your best agent play against the other student's best player. We will set up a class tournament on Canvas. This tournament will continue after the submission deadline.

## Graduate student advanced task: Pure Monte Carlo Search and Best First Move [1 point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+1 Bonus point].

### Pure Monte Carlos Search

Implement Pure Monte Carlo Search and investigate how this search performs on the test boards that you have used above. 

In [17]:
# Your code/ answer goes here.

### Best First Move

How would you determine what the best first move is? You can use Pure Monte Carlo Search or any algorithms 
that you have implemented above.

In [18]:
# Your code/ answer goes here.

In [160]:
def basic_utility(board, player='x'):
    series = get_all_series(board)
    util = 0
    for s in series:
        as_string = ""
        as_string = as_string.join(s)
        # points for xxx
        
        idx = as_string.find('xxx')
        while idx >= 0:
            if idx-1 >= 0 and as_string[idx-1] == ' ':
                util += 3
            if idx >= 0 and idx+3 < len(as_string) and as_string[idx+3] == ' ':
                util += 3
            print(idx, as_string, util)
            idx = as_string.find('xxx', idx+3)
            
        #points for xx
        idx = as_string.find('xx')
        while idx >= 0:
            if idx-1 >= 0 and as_string[idx-1] == player:
                idx = as_string.find('xx', idx+2)
                continue
            if idx+2 < len(as_string) and as_string[idx+2] == player:
                idx = as_string.find('xx', idx+2)
                continue
            if idx-1 >= 0 and as_string[idx-1] == ' ':
                util += 2
            if idx > 0 and idx+2 < len(as_string) and as_string[idx+2] == ' ':
                util += 2
            idx = as_string.find('xx', idx+2)
    return util